In [261]:
%run input/Format.ipynb
import ROOT as root
from array import array
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

In [262]:
shift = 0
antiproton = 1

In [263]:
N_centr = 6
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
central_bins = [0,100,0,20,20,40,40,60,60,80,80,100]
pt_boarders = [0.5,0.9,0.2,0.5,0.5,1.0,1.0,5.0,0.5,5]
file_path="input/"
file_names=["my-m_ee_Run14AuAu_46th_new_19619_999runs.root"] #m_ee_Run14AuAu_46th_new_19619_90runs test_BG_v94_0
sim_file_path="../sim/output/Newembed/sngl/"
primary_file_names=['phi_50M_v6.root']
hadron_file_names=['charged_pi_100M_v1.root']
dalitz_file_names=['dalitz_50M_v1.root']
conv_file_names=['conv_100M.root']
sim_file_names = [dalitz_file_names[0]]
legend_names = ["pos","neg","sim_pos","sim_neg"] 
hist_select_3D_names = ["ep_hist","n0_hist","prob_hist","disp_hist","chi2npe0_hist","ttof","temc",\
                        "el_had_dphi","el_had_dz","el_had_dr",'disp_hist_el']

In [264]:
N_hists = len(hist_select_3D_names)

In [265]:
hists_read, hists_sim = [], []

for iFile in range(len(file_names)):
    print(file_path+file_names[iFile])
    infile = root.TFile.Open(file_path+file_names[iFile], "read")
    hists_read_file = []
    for ihist in range(N_hists):
        hists_read_file.append(infile.Get(hist_select_3D_names[ihist]))
        hists_read_file[-1].SetDirectory(root.nullptr)
        hists_read_file[-1].SetName(hists_read_file[-1].GetName()+f"{iFile}")
    hists_read.append(hists_read_file)


for iFile in range(len(sim_file_names)):
    print(sim_file_path+sim_file_names[iFile])
    infile = root.TFile.Open(sim_file_path+sim_file_names[iFile], "read")
    hists_sim_file = []
    for ihist in range(N_hists):
        hists_sim_file.append(infile.Get(hist_select_3D_names[ihist]))
        hists_sim_file[-1].SetDirectory(root.nullptr)
        hists_sim_file[-1].SetName(hists_sim_file[-1].GetName()+f"_sim_{iFile}")
    hists_sim.append(hists_sim_file)

infile.Close()

input/my-m_ee_Run14AuAu_46th_new_19619_999runs.root
../sim/output/Newembed/sngl/dalitz_50M_v1.root


In [266]:
if shift<20: 
    if shift< 10 : antiproton = antiproton*15
    else : andiproton = antiproton*5
    if antiproton>0: 
        shift += 5
        legend_names = ["antip","neg","sim_antip","sim_neg"] 
else: antiproton = 0

In [267]:
type_loc = 0
pt = [1.95,2.45]
c0 = root.TCanvas(f"c0",f"c0",1450,900)
c0.Divide(3,2)
projy = hists_read[0][type_loc].ProjectionY(f"projy{type_loc}")
projz = hists_read[0][type_loc].ProjectionZ(f"projz{type_loc}")
legends = []
for icentr in range(N_centr):
    c0.cd(1+icentr)
    ibiny0 = projy.FindBin(pt[0])
    ibiny1 = projy.FindBin(pt[1])-1
    ibinz0 = shift+projz.FindBin(central_bins[2*icentr+0])
    ibinz1 = shift+projz.FindBin(central_bins[2*icentr+1])-1
    pos_proj = hists_read[iFile][type_loc].ProjectionX(hists_read[iFile][type_loc].GetName()+f"pos_proj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(pos_proj,title_x="E/p",offset_y=1.1)
    neg_proj = hists_read[iFile][type_loc].ProjectionX(hists_read[iFile][type_loc].GetName()+f"neg_proj_{icentr}",ibiny0,ibiny1,ibinz0+5+antiproton,ibinz1+5+antiproton)
    Format_Hist_total(neg_proj,Lcolor=2)
    sim_pos_proj = hists_sim[iFile][type_loc].ProjectionX(hists_sim[iFile][type_loc].GetName()+f"sim_pos_proj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(sim_pos_proj,Lcolor=root.kGreen+2)
    sim_neg_proj = hists_sim[iFile][type_loc].ProjectionX(hists_sim[iFile][type_loc].GetName()+f"sim_neg_proj){icentr}",ibiny0,ibiny1,ibinz0+5+antiproton,ibinz1+5+antiproton)
    Format_Hist_total(sim_neg_proj,Lcolor=1)
    pos_proj.Scale(1./(1+pos_proj.GetMaximum()))
    neg_proj.Scale(1./(1+neg_proj.GetMaximum()))
    sim_pos_proj.Scale(1./(1+sim_pos_proj.GetMaximum()))
    sim_neg_proj.Scale(1./(1+sim_neg_proj.GetMaximum()))
    pos_proj.Draw()
    neg_proj.Draw("same")
    sim_pos_proj.Draw("same")
    sim_neg_proj.Draw("same")
    legends.append(root.TLegend(0.2,0.6,0.65,0.95,f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%, pT={pt[0]}"))
    legends[-1].AddEntry(pos_proj,legend_names[0],"l")
    legends[-1].AddEntry(neg_proj,legend_names[1],"l")
    legends[-1].AddEntry(sim_pos_proj,legend_names[2],"l")
    legends[-1].AddEntry(sim_neg_proj,legend_names[3],"l")
    Format_Legend(legends[-1])
    legends[-1].Draw()

c0.Draw()
c0.SaveAs(f"output/rich_emcal_qa/qa_pt_centr_{icentr}.png")

In [268]:
type_loc = 2
pt = [1.95,2.45]
c0 = root.TCanvas(f"c0",f"c0",1450,900)
c0.Divide(3,2)
projy = hists_read[0][type_loc].ProjectionY(f"projy{type_loc}")
projz = hists_read[0][type_loc].ProjectionZ(f"projz{type_loc}")
legends = []
for icentr in range(N_centr):
    c0.cd(1+icentr)
    ibiny0 = projy.FindBin(pt[0])
    ibiny1 = projy.FindBin(pt[1])-1
    ibinz0 = shift+projz.FindBin(central_bins[2*icentr+0])
    ibinz1 = shift+projz.FindBin(central_bins[2*icentr+1])-1
    pos_proj = hists_read[iFile][type_loc].ProjectionX(hists_read[iFile][type_loc].GetName()+f"pos_proj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(pos_proj,title_x="prob")
    neg_proj = hists_read[iFile][type_loc].ProjectionX(hists_read[iFile][type_loc].GetName()+f"neg_proj_{icentr}",ibiny0,ibiny1,ibinz0+5+antiproton,ibinz1+5+antiproton)
    Format_Hist_total(neg_proj,Lcolor=2,title_x="prob",offset_y=1.2)
    sim_pos_proj = hists_sim[iFile][type_loc].ProjectionX(hists_sim[iFile][type_loc].GetName()+f"sim_pos_proj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(sim_pos_proj,Lcolor=root.kGreen+2)
    sim_neg_proj = hists_sim[iFile][type_loc].ProjectionX(hists_sim[iFile][type_loc].GetName()+f"sim_neg_proj){icentr}",ibiny0,ibiny1,ibinz0+5+antiproton,ibinz1+5+antiproton)
    Format_Hist_total(sim_neg_proj,Lcolor=1)
    pos_proj.Scale(1./(1+pos_proj.Integral()))
    neg_proj.Scale(1./(1+neg_proj.Integral()))
    sim_pos_proj.Scale(1./(1+sim_pos_proj.Integral()))
    sim_neg_proj.Scale(1./(1+sim_neg_proj.Integral()))
    neg_proj.SetMaximum(max(neg_proj.GetMaximum(),pos_proj.GetMaximum()))
    neg_proj.Draw()
    pos_proj.Draw("same")
    sim_pos_proj.Draw("same")
    sim_neg_proj.Draw("same")
    legends.append(root.TLegend(0.2,0.6,0.65,0.95,f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%, pT={pt[0]}"))
    legends[-1].AddEntry(pos_proj,legend_names[0],"l")
    legends[-1].AddEntry(neg_proj,legend_names[1],"l")
    legends[-1].AddEntry(sim_pos_proj,legend_names[2],"l")
    legends[-1].AddEntry(sim_neg_proj,legend_names[3],"l")
    Format_Legend(legends[-1])
    legends[-1].Draw()

c0.Draw()
c0.SaveAs(f"output/rich_emcal_qa/qa_pt_centr_{icentr}.png")

In [269]:
type_loc = 1
pt = [1.95,2.45]
c0 = root.TCanvas(f"c0",f"c0",1450,900)
c0.Divide(3,2)
projy = hists_read[0][type_loc].ProjectionY(f"projy{type_loc}")
projz = hists_read[0][type_loc].ProjectionZ(f"projz{type_loc}")
legends = []
for icentr in range(N_centr):
    c0.cd(1+icentr)
    ibiny0 = projy.FindBin(pt[0])
    ibiny1 = projy.FindBin(pt[1])-1
    ibinz0 = shift+projz.FindBin(central_bins[2*icentr+0])
    ibinz1 = shift+projz.FindBin(central_bins[2*icentr+1])-1
    pos_proj = hists_read[iFile][type_loc].ProjectionX(hists_read[iFile][type_loc].GetName()+f"pos_proj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(pos_proj,title_x="n0",offset_y=1.18)
    neg_proj = hists_read[iFile][type_loc].ProjectionX(hists_read[iFile][type_loc].GetName()+f"neg_proj_{icentr}",ibiny0,ibiny1,ibinz0+5+antiproton,ibinz1+5+antiproton)
    Format_Hist_total(neg_proj,Lcolor=2)
    sim_pos_proj = hists_sim[iFile][type_loc].ProjectionX(hists_sim[iFile][type_loc].GetName()+f"sim_pos_proj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(sim_pos_proj,Lcolor=root.kGreen+2)
    sim_neg_proj = hists_sim[iFile][type_loc].ProjectionX(hists_sim[iFile][type_loc].GetName()+f"sim_neg_proj){icentr}",ibiny0,ibiny1,ibinz0+5+antiproton,ibinz1+5+antiproton)
    Format_Hist_total(sim_neg_proj,Lcolor=1)
    pos_proj.Scale(1./(1+pos_proj.Integral()))
    neg_proj.Scale(1./(1+neg_proj.Integral()))
    sim_pos_proj.Scale(1./(1+sim_pos_proj.Integral()))
    sim_neg_proj.Scale(1./(1+sim_neg_proj.Integral()))
    pos_proj.SetMaximum(max(neg_proj.GetMaximum(),pos_proj.GetMaximum(),sim_pos_proj.GetMaximum(),sim_neg_proj.GetMaximum()))
    pos_proj.Draw()
    neg_proj.Draw("same")
    sim_pos_proj.Draw("same")
    sim_neg_proj.Draw("same")
    legends.append(root.TLegend(0.15,0.58,0.45,0.98,f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%, pT={pt[0]}"))
    legends[-1].AddEntry(pos_proj,legend_names[0],"l")
    legends[-1].AddEntry(neg_proj,legend_names[1],"l")
    legends[-1].AddEntry(sim_pos_proj,legend_names[2],"l")
    legends[-1].AddEntry(sim_neg_proj,legend_names[3],"l")
    Format_Legend(legends[-1])
    legends[-1].Draw()

c0.Draw()
c0.SaveAs(f"output/rich_emcal_qa/qa_pt_centr_{icentr}.png")

In [270]:
type_loc = 3
pt = [1.95,2.45]
c0 = root.TCanvas(f"c0",f"c0",1450,900)
c0.Divide(3,2)
projy = hists_read[0][type_loc].ProjectionY(f"projy{type_loc}")
projz = hists_read[0][type_loc].ProjectionZ(f"projz{type_loc}")
legends = []
for icentr in range(N_centr):
    c0.cd(1+icentr)
    ibiny0 = projy.FindBin(pt[0])
    ibiny1 = projy.FindBin(pt[1])-1
    ibinz0 = shift+projz.FindBin(central_bins[2*icentr+0])
    ibinz1 = shift+projz.FindBin(central_bins[2*icentr+1])-1
    pos_proj = hists_read[iFile][type_loc].ProjectionX(hists_read[iFile][type_loc].GetName()+f"pos_proj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(pos_proj,title_x="disp",offset_y=1.2)
    neg_proj = hists_read[iFile][type_loc].ProjectionX(hists_read[iFile][type_loc].GetName()+f"neg_proj_{icentr}",ibiny0,ibiny1,ibinz0+5+antiproton,ibinz1+5+antiproton)
    Format_Hist_total(neg_proj,Lcolor=2)
    sim_pos_proj = hists_sim[iFile][type_loc].ProjectionX(hists_sim[iFile][type_loc].GetName()+f"sim_pos_proj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(sim_pos_proj,Lcolor=root.kGreen+2)
    sim_neg_proj = hists_sim[iFile][type_loc].ProjectionX(hists_sim[iFile][type_loc].GetName()+f"sim_neg_proj){icentr}",ibiny0,ibiny1,ibinz0+5+antiproton,ibinz1+5+antiproton)
    Format_Hist_total(sim_neg_proj,Lcolor=1)
    pos_proj.Scale(1./(1+pos_proj.Integral()))
    neg_proj.Scale(1./(1+neg_proj.Integral()))
    sim_pos_proj.Scale(1./(1+sim_pos_proj.Integral()))
    sim_neg_proj.Scale(1./(1+sim_neg_proj.Integral()))
    pos_proj.SetMaximum(max(neg_proj.GetMaximum(),pos_proj.GetMaximum(),sim_pos_proj.GetMaximum(),sim_neg_proj.GetMaximum()))
    pos_proj.Draw()
    neg_proj.Draw("same")
    sim_pos_proj.Draw("same")
    sim_neg_proj.Draw("same")
    legends.append(root.TLegend(0.55,0.55,0.95,0.95,f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%, pT={pt[0]}"))
    legends[-1].AddEntry(pos_proj,legend_names[0],"l")
    legends[-1].AddEntry(neg_proj,legend_names[1],"l")
    legends[-1].AddEntry(sim_pos_proj,legend_names[2],"l")
    legends[-1].AddEntry(sim_neg_proj,legend_names[3],"l")
    Format_Legend(legends[-1])
    legends[-1].Draw()

c0.Draw()
c0.SaveAs(f"output/rich_emcal_qa/qa_pt_centr_{icentr}.png")

In [271]:
type_loc = 4
pt = [1.95,2.45]
c0 = root.TCanvas(f"c0",f"c0",1450,900)
c0.Divide(3,2)
projy = hists_read[0][type_loc].ProjectionY(f"projy{type_loc}")
projz = hists_read[0][type_loc].ProjectionZ(f"projz{type_loc}")
legends = []
for icentr in range(N_centr):
    c0.cd(1+icentr)
    ibiny0 = projy.FindBin(pt[0])
    ibiny1 = projy.FindBin(pt[1])-1
    ibinz0 = shift+projz.FindBin(central_bins[2*icentr+0])
    ibinz1 = shift+projz.FindBin(central_bins[2*icentr+1])-1
    pos_proj = hists_read[iFile][type_loc].ProjectionX(hists_read[iFile][type_loc].GetName()+f"pos_proj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(pos_proj,title_x="#chi^{2}/npe0")
    neg_proj = hists_read[iFile][type_loc].ProjectionX(hists_read[iFile][type_loc].GetName()+f"neg_proj_{icentr}",ibiny0,ibiny1,ibinz0+5+antiproton,ibinz1+5+antiproton)
    Format_Hist_total(neg_proj,Lcolor=2)
    sim_pos_proj = hists_sim[iFile][type_loc].ProjectionX(hists_sim[iFile][type_loc].GetName()+f"sim_pos_proj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(sim_pos_proj,Lcolor=root.kGreen+2)
    sim_neg_proj = hists_sim[iFile][type_loc].ProjectionX(hists_sim[iFile][type_loc].GetName()+f"sim_neg_proj){icentr}",ibiny0,ibiny1,ibinz0+5+antiproton,ibinz1+5+antiproton)
    Format_Hist_total(sim_neg_proj,Lcolor=1)
    pos_proj.Scale(1./(1+pos_proj.GetMaximum()))
    neg_proj.Scale(1./(1+neg_proj.GetMaximum()))
    sim_pos_proj.Scale(1./(1+sim_pos_proj.GetMaximum()))
    sim_neg_proj.Scale(1./(1+sim_neg_proj.GetMaximum()))
    pos_proj.Draw()
    neg_proj.Draw("same")
    sim_pos_proj.Draw("same")
    sim_neg_proj.Draw("same")
    legends.append(root.TLegend(0.5,0.55,0.95,0.95,f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%, pT={pt[0]}"))
    legends[-1].AddEntry(pos_proj,legend_names[0],"l")
    legends[-1].AddEntry(neg_proj,legend_names[1],"l")
    legends[-1].AddEntry(sim_pos_proj,legend_names[2],"l")
    legends[-1].AddEntry(sim_neg_proj,legend_names[3],"l")
    Format_Legend(legends[-1])
    legends[-1].Draw()

c0.Draw()
c0.SaveAs(f"output/rich_emcal_qa/qa_pt_centr_{icentr}.png")

In [272]:
type_loc = 5
pt = [1.95,2.45]
c0 = root.TCanvas(f"c0",f"c0",1450,900)
c0.Divide(3,2)
projy = hists_read[0][type_loc].ProjectionY(f"projy{type_loc}")
projz = hists_read[0][type_loc].ProjectionZ(f"projz{type_loc}")
legends = []
for icentr in range(N_centr):
    c0.cd(1+icentr)
    ibiny0 = projy.FindBin(pt[0])
    ibiny1 = projy.FindBin(pt[1])-1
    ibinz0 = shift+projz.FindBin(central_bins[2*icentr+0])
    ibinz1 = shift+projz.FindBin(central_bins[2*icentr+1])-1
    pos_proj = hists_read[iFile][type_loc].ProjectionX(f"pos_proj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(pos_proj,title_x="m_tof")
    neg_proj = hists_read[iFile][type_loc].ProjectionX(f"neg_proj_{icentr}",ibiny0,ibiny1,ibinz0+5+antiproton,ibinz1+5+antiproton)
    Format_Hist_total(neg_proj,Lcolor=2,title_x="m2_tof",offset_y=1.2)
    sim_pos_proj = hists_sim[iFile][type_loc].ProjectionX(f"sim_pos_proj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(sim_pos_proj,Lcolor=root.kGreen+2)
    sim_neg_proj = hists_sim[iFile][type_loc].ProjectionX(f"sim_neg_proj){icentr}",ibiny0,ibiny1+5)
    pos_proj.Rebin(10)
    neg_proj.Rebin(10)
    Format_Hist_total(sim_neg_proj,Lcolor=1)
    pos_proj.Scale(1./(1+pos_proj.Integral()))
    neg_proj.Scale(1./(1+neg_proj.Integral()))
    root.gPad.SetLogy()
    neg_proj.Draw()
    pos_proj.SetLineStyle(2)
    pos_proj.Draw("same")
    #sim_pos_proj.Draw("same")
    #sim_neg_proj.Draw("same")
    legends.append(root.TLegend(0.5,0.65,0.99,0.95,f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%, pT={pt[0]}"))
    legends[-1].AddEntry(pos_proj,"pos","l")
    legends[-1].AddEntry(neg_proj,"neg","l")
    #legends[-1].AddEntry(sim_pos_proj,"sim pos","l")
    #legends[-1].AddEntry(sim_neg_proj,"sim neg","l")
    Format_Legend(legends[-1])
    legends[-1].Draw()
    print(round(100*pos_proj.Integral(pos_proj.FindBin(0.7),pos_proj.FindBin(1.2))/pos_proj.Integral(pos_proj.FindBin(-0.3),pos_proj.FindBin(0.4)),1),\
          round(100*neg_proj.Integral(neg_proj.FindBin(0.7),neg_proj.FindBin(1.2))/neg_proj.Integral(neg_proj.FindBin(-0.3),neg_proj.FindBin(0.4)),1))

c0.Draw()
c0.SaveAs(f"output/rich_emcal_qa/qa_pt_centr_{icentr}.png")

5.8 10.6
7.8 14.5
4.2 7.3
2.7 3.8
1.7 2.0
1.5 1.5


In [273]:
type_loc = 5
c0 = root.TCanvas(f"c0",f"c0",1450,900)
c0.Divide(3,2)
projy = hists_read[0][type_loc].ProjectionY(f"projy{type_loc}")
projz = hists_read[0][type_loc].ProjectionZ(f"projz{type_loc}")
graphs_neg, graphs_pos = [], []
legends = []
pt_bins = [0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.25,1.5,2.0,2.5,3.0,3.5]
for icentr in range(N_centr):
    c0.cd(1+icentr)
    ibinz0 = shift+projz.FindBin(central_bins[2*icentr+0])
    ibinz1 = shift+projz.FindBin(central_bins[2*icentr+1])-1
    graphs_pos.append(root.TGraph())
    graphs_neg.append(root.TGraph())
    for ipt in range(len(pt_bins)-1):
        ibiny0 = projy.FindBin(pt_bins[ipt])
        ibiny1 = projy.FindBin(pt_bins[ipt])
        pos_proj = hists_read[iFile][type_loc].ProjectionX(f"pos_pt_proj_{icentr}_{ipt}",ibiny0,ibiny1,ibinz0,ibinz1)
        neg_proj = hists_read[iFile][type_loc].ProjectionX(f"neg_pt_proj_{icentr}_{ipt}",ibiny0,ibiny1,ibinz0+5+antiproton,ibinz1+5+antiproton)
        graphs_pos[-1].AddPoint( (pt_bins[ipt] + pt_bins[ipt+1] )/2,  pos_proj.Integral(pos_proj.FindBin(0.7),pos_proj.FindBin(1.4))/(1+pos_proj.Integral(pos_proj.FindBin(-0.3),pos_proj.FindBin(0.4))))
        graphs_neg[-1].AddPoint( (pt_bins[ipt] + pt_bins[ipt+1] )/2,  neg_proj.Integral(neg_proj.FindBin(0.7),neg_proj.FindBin(1.4))/(1+neg_proj.Integral(neg_proj.FindBin(-0.3),neg_proj.FindBin(0.4))))
    Format_Hist_total(graphs_pos[-1], Msize=1.5)
    Format_Hist_total(graphs_neg[-1], Msize=1.5, Mcolor=2,offset_y=1.2)
    graphs_neg[-1].Draw("AP")
    graphs_pos[-1].Draw("sameP")
    legends.append(root.TLegend(0.2,0.7,0.5,0.95,f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%"))
    legends[-1].AddEntry(graphs_neg[-1],"neg","p")
    legends[-1].AddEntry(graphs_pos[-1],"pos","p")
    Format_Legend(legends[-1])
    legends[-1].Draw()

c0.Draw()
c0.SaveAs(f"output/rich_emcal_qa/qa_pt_centr_{icentr}.png")

In [274]:
type_loc = 6
pt = [1.45,2.45]
c0 = root.TCanvas(f"c0",f"c0",1450,900)
c0.Divide(3,2)
projy = hists_read[0][type_loc].ProjectionY(f"projy{type_loc}")
projz = hists_read[0][type_loc].ProjectionZ(f"projz{type_loc}")
legends = []
for icentr in range(N_centr):
    c0.cd(1+icentr)
    ibiny0 = projy.FindBin(pt[0])
    ibiny1 = projy.FindBin(pt[1])-1
    ibinz0 = shift+projz.FindBin(central_bins[2*icentr+0])
    ibinz1 = shift+projz.FindBin(central_bins[2*icentr+1])-1
    pos_proj = hists_read[iFile][type_loc].ProjectionX(f"pos_proj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(pos_proj,title_x="temc")
    neg_proj = hists_read[iFile][type_loc].ProjectionX(f"neg_proj_{icentr}",ibiny0,ibiny1,ibinz0+5+antiproton,ibinz1+5+antiproton)
    Format_Hist_total(neg_proj,Lcolor=2)
    sim_pos_proj = hists_sim[iFile][type_loc].ProjectionX(f"sim_pos_proj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(sim_pos_proj,Lcolor=root.kGreen+2)
    sim_neg_proj = hists_sim[iFile][type_loc].ProjectionX(f"sim_neg_proj){icentr}",ibiny0,ibiny1,ibinz0+5+antiproton,ibinz1+5+antiproton)
    Format_Hist_total(sim_neg_proj,Lcolor=1)
    pos_proj.Scale(1./(1+pos_proj.GetMaximum()))
    neg_proj.Scale(1./(1+neg_proj.GetMaximum()))
    sim_pos_proj.Scale(1./(1+sim_pos_proj.GetMaximum()))
    sim_neg_proj.Scale(1./(1+sim_neg_proj.GetMaximum()))
    pos_proj.Draw()
    neg_proj.Draw("same")
    #sim_pos_proj.Draw("same")
    #sim_neg_proj.Draw("same")
    legends.append(root.TLegend(0.5,0.5,0.9,0.9,f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%"))
    legends[-1].AddEntry(pos_proj,"pos","l")
    legends[-1].AddEntry(neg_proj,"neg","l")
    #legends[-1].AddEntry(sim_pos_proj,"sim pos","l")
    #legends[-1].AddEntry(sim_neg_proj,"sim neg","l")
    Format_Legend(legends[-1])
    legends[-1].Draw()

c0.Draw()
c0.SaveAs(f"output/rich_emcal_qa/qa_pt_centr_{icentr}.png")

In [275]:
type_loc = 10
pt = [1.45,2.45]
c0 = root.TCanvas(f"c0",f"c0",1450,900)
c0.Divide(3,2)
projx = hists_read[0][type_loc].ProjectionY(f"projy{type_loc}")
projz = hists_read[0][type_loc].ProjectionZ(f"projz{type_loc}")
legends = []
for icentr in range(N_centr):
    c0.cd(1+icentr)
    ibinx0 = projx.FindBin(0)
    ibinx1 = projx.FindBin(10)-1
    ibinz0 = shift+projz.FindBin(central_bins[2*icentr+0])
    ibinz1 = shift+projz.FindBin(central_bins[2*icentr+1])-1
    pos_proj = hists_read[iFile][type_loc].ProjectionY(f"pos_proj_{icentr}",ibinx0,ibinx1,ibinz0,ibinz1)
    Format_Hist_total(pos_proj,title_x="npe0")
    sim_pos_proj = hists_sim[iFile][type_loc].ProjectionY(f"sim_pos_proj_{icentr}",ibinx0,ibinx1,ibinz0,ibinz1)
    Format_Hist_total(sim_pos_proj,Lcolor=root.kGreen+2)
    pos_proj.Scale(1./(1+pos_proj.GetMaximum()))
    sim_pos_proj.Scale(1./(1+sim_pos_proj.GetMaximum()))
    pos_proj.Draw()
    sim_pos_proj.Draw("same")
    legends.append(root.TLegend(0.5,0.5,0.9,0.9,f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%"))
    legends[-1].AddEntry(pos_proj,"data","l")
    legends[-1].AddEntry(sim_pos_proj,"sim","l")
    Format_Legend(legends[-1])
    legends[-1].Draw()

c0.Draw()
c0.SaveAs(f"output/rich_emcal_qa/qa_pt_centr_{icentr}.png")